<a href="https://colab.research.google.com/github/VOX304/BasicChatbot/blob/main/Preprocessing/ImgPrep/Gemini_Images_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dowload packages

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [ ]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")


# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install pillow

In [ ]:
!pip install -q google google.genai pillow faiss-cpu open_clip_torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.

In [ ]:
!pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=3c560e2a2d6a69eec35b1d73466da15f70b195f5eaa5fcd82140d9aa36a73233
  Stored in directory: /root/.cache/pip/wheels/0d/17/90/042948fd2e2a87f1dcf6db6d438cad015c49db0c53d1d9c7dc
Successfully built openai-clip


# Image Prep

In [ ]:
from PIL import Image
import io
import base64
import os
from google import genai
from google.genai import types

def image_to_base64(image_path):
    """Convert an image file to a base64-encoded string."""
    with Image.open(image_path) as image:
        buffered = io.BytesIO()
        image.save(buffered, format=image.format)
        img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')
    return img_str

def extract_image_content(image_path):
    """Extract image content description using Gemini."""
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )
    image_str = image_to_base64(image_path)
    contents = [{
        "role": "user",
        "parts": [
            {"text": "Vui lòng tóm tắt nội dung hình ảnh này.Thường sẽ liên quan đến Quân Đội"},
            {"inline_data": {"mime_type": "image/jpeg", "data": image_str}}
        ],
    }]

    response = client.models.generate_content(
        model="gemini-2.0-pro-exp-02-05",
        contents=contents
    )

    return response.text.strip()

def convert_images_in_folder(folder_path):
    """Convert all images in a folder to base64 strings and extract their content."""
    image_data = {}

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if os.path.isfile(file_path) and filename.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
            image_content = extract_image_content(file_path)
            image_data[filename] = image_content

    return image_data

# Specify the folder containing images
folder_path = "/content/sample_data/TCU"
image_content_dict = convert_images_in_folder(folder_path)

# Example: Print extracted content for one image
for img_name, img_content in image_content_dict.items():
    print(f"{img_name}: {img_content}")



KeyboardInterrupt



In [ ]:
for img_name, img_content in image_content_dict.items():
    print(f"{img_name}: {img_content}")

Picture5.jpg: Okay, đây là tóm tắt nội dung hình ảnh:

Hình ảnh là ảnh chụp màn hình của một kênh YouTube có tên **"Tự hào Sĩ quan Thông tin"**.

*   **Đây là kênh YouTube chính thức của Trường Sĩ quan Thông tin**, thuộc Binh chủng Thông tin liên lạc - Bộ Quốc phòng.
*   Kênh có **9,06 nghìn người đăng ký** và đã đăng tải **719 video**.
*   Mục đích của kênh là **cập nhật các clip, tin tức** liên quan đến nhà trường, hoạt động của sĩ quan thông tin và có thể là các tin tức thời sự, chính trị liên quan đến quân đội.

Như bạn đã đề cập, nội dung rõ ràng liên quan đến **Quân đội Nhân dân Việt Nam**, cụ thể là **Trường Sĩ quan Thông tin** và **Binh chủng Thông tin liên lạc**.
Picture3.jpg: Chào bạn, đây là tóm tắt nội dung hình ảnh:

Hình ảnh này là một ảnh chụp màn hình giao diện Facebook, cho thấy:

1.  **Banner ở trên:** Là một áp phích tuyên truyền kỷ niệm 95 năm ngày thành lập Đảng Cộng sản Việt Nam (3/2/1930 - 3/2/2025). Banner có hình ảnh Chủ tịch Hồ Chí Minh, biểu tượng búa liềm và

In [ ]:
import json

# Convert dictionary into a list of dictionaries
image_metadata = [
    { "document": "Địa chỉ trang mạng_2025313935.docx", "file_name": img_name, "description": img_content}
    for img_name, img_content in image_content_dict.items()
]

# Save to JSON file
json_file_path = "image_metadata.json"
with open(json_file_path, "w", encoding="utf-8") as f:
    json.dump(image_metadata, f, ensure_ascii=False, indent=4)

print(f"✅ Image metadata saved to {json_file_path}")


✅ Image metadata saved to image_metadata.json


# Embedding

In [ ]:
import json
import os
import base64
import numpy as np
import torch
import clip
from PIL import Image
from google import genai
from google.genai import types
import faiss

# Load CLIP model for image embeddings
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 74.0MiB/s]


In [ ]:


# Function to extract image content using Gemini
def get_image_description(image_path):
    """Use Gemini to extract textual content from an image"""
    with open(image_path, "rb") as img_file:
        image_bytes = img_file.read()
        image_base64 = base64.b64encode(image_bytes).decode("utf-8")

    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
    model_name = "gemini-2.0-flash-thinking-exp-01-21"

    contents = [
        {"role": "user", "parts": [
            {"text": "Vui lòng tóm tắt nội dung hình ảnh này.Thường sẽ liên quan đến Quân Đội"},
            {"inline_data": {"mime_type": "image/jpeg", "data": image_base64}}
        ]}
    ]

    response = client.models.generate_content(model=model_name, contents=contents)
    return response.text if response else "No description available."

# Function to generate image embeddings using CLIP
def get_image_embedding(image_path):
    """Generate image embeddings using CLIP"""
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image)
    return embedding.cpu().numpy().flatten()

# Function to embed text descriptions using CLIP
def get_text_embedding(text):
    """Convert a text description into a CLIP embedding"""
    text_tokens = clip.tokenize([text]).to(device)
    with torch.no_grad():
        embedding = model.encode_text(text_tokens)
    return embedding.cpu().numpy().flatten()

# Process images and store text embeddings
def process_images(image_folder, json_output):
    image_metadata = []

    for img_name in os.listdir(image_folder):
        if not img_name.lower().endswith((".jpg", ".png", ".jpeg")):
            continue  # Skip non-image files

        img_path = os.path.join(image_folder, img_name)
        description = get_image_description(img_path)

        image_metadata.append({
            "document_name": "Địa chỉ trang mạng_2025313935.docx",
            "datatype": "img",
            "img_path": img_path,
            "extracted_content": description,
        })

    # Save metadata and embeddings to JSON
    with open(json_output, "w", encoding="utf-8") as f:
        json.dump(image_metadata, f, ensure_ascii=False, indent=4)

    return image_metadata

# Example usage
image_folder = "/content/sample_data/TCU"
json_output = "image_metadata.json"
image_data = process_images(image_folder, json_output)
print("✅ Image descriptions & embeddings saved.")


✅ Image descriptions & embeddings saved.
